<a target="_blank" href="https://colab.research.google.com/github/taobrienlbl/colab_notebooks_for_atmos_sci/blob/main/Baroclinic%20Wave%20Structure.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Observed Baroclinic Waves
This [Notebook](https://jupyterlab.readthedocs.io/en/stable/user/notebook.html) aids in exploring the structure of [Rossby waves](https://oceanservice.noaa.gov/facts/rossby-wave.html) in real observational data (specifically, [the NCEP/NCAR Reanalysis I](https://psl.noaa.gov/data/gridded/data.ncep.reanalysis.html))

**Objective**:

* Describe the horizontal and vertical structure of real baroclinic Rossby waves

**Instructions**:

1. Run the code in Cells 1, 2, 3, and 4.  Cell 4 should generate a map if successful.
2. Choose a different date than shown in Cell 3; find a good example of a Rossby wave (you may have to run cells 3 and 4 several times to find a good date)
3. Uncomment the indicated line in Cell 3 to download all the variables, and re-run Cell 3 (it'll take several minutes)
4. Run cells 4 and 5
5. Examine the transect and comment on the structure.  If this is part of an in-class exercise, complete the exercise.

Note that cells 1 and 3 may take a minute or two to run (especially once Cell 3 has been modified to download multiple datasets).

## A note on 'notebooks'
One thing to point out is that there are two types of text in the cells below: comment text and code text.  Comment text doesn't actually do anything--it's just meant to help a reader understand the code.  Comment text shows up as one of two things:

```python
""" This is a comment in triple-quotes.  I often use this to say what the cell is meant to do. """

# this is a traditional comment; anything that comes after the # symbol on a given line counts as a comment
```

**NOTE:** when you want to "run" the code in any of the cells below, click on the cell and then press the "play" button in the toolbar above...or press Shift+Enter on your keyboard.

In [ ]:
""" Cell 1: Install packages in a special way if in Google Colab. """
# ******************************************************************************
# NOTE: the following code is overly complicated because we are using Google 
# Colab, which does not have several of the packages we want installed.
# If you are running this in your own conda environment, these things install
# simply - this step isn't necessary.
# ******************************************************************************
# check if we are in google colab
# (skip everything in this cell if not)
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False


if IN_COLAB:
  # upgrade pip to avoid pip auto-updating packages that don't need updating
  ! pip install --upgrade pip
  # install aiohttp to allow remote access to NOAA data
  try:
    import aiohttp
  except ImportError:
    ! pip install aiohttp

  try:
    import cartopy
  except ImportError:
    # force re-installation of shapely, otherwise cartopy breaks in google colab
    ! pip uninstall shapely --yes
    # install cartopy and shapely
    ! pip install --no-binary shapely cartopy


In [ ]:
""" Cell 2: Import libraries. """

import fsspec # for accessing remote data
import xarray as xr # for loading reanalysis data
import matplotlib as mpl # for plotting 
import matplotlib.pyplot as plt # also for plotting
import cartopy # for drawing maps on plots
import seaborn as sns # for making plot look pretty
import datetime as dt # for manipulating dates
import numpy as np # for doing math and working with arrays

# use seaborn to make the plots look nice
sns.set_theme(style = "whitegrid")

print("Libraries imported successfully!")

In [ ]:
""" Cell 3: Access reanalysis data for the current date. """

#**********************
# Step 1: set the date
#**********************

#---------------------------------------------------------------------
# Cell 3 subset: modify the following line to change the date plotted
#---------------------------------------------------------------------
plot_date = dt.datetime(
    year = 2023,
    month = 4, 
    day = 12,
    hour = 0, # note: valid hours are 0, 6, 12, and 18 - these are in UTC
)

# ***********************************
# Step 2: Access the reanalysis data
# ***********************************
# the pattern for URLs for accessing reanalysis data from NOAA
url_template = "https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis/pressure//{variable}.{year}.nc"

# set the variables we want to grab
variables = ["hgt"] # for now, just grab the geopotential height 

#---------------------------------------------------------------------
# Cell 3 subset: uncomment the following lines to grab more variables
#---------------------------------------------------------------------
# note: later in the exercise, we will grab the following variables by uncommenting the line below (i.e., remove the # at the beginning of the line)
#
#variables = ["air", "hgt", "uwnd", "omega"] # get everything

# initialize the reanalysis dataset
reanalysis_xr = None

# create a dictionary to store remote file handles - this
# avoids redownloading the same file multiple times
file_handles = {var : {} for var in variables}
for var in variables:
    if plot_date.year not in file_handles[var]:
        file_handles[var][plot_date.year] = fsspec.open(url_template.format(variable = var, year = plot_date.year), )

    with file_handles[var][plot_date.year] as f:
        tmp_xr = xr.open_dataset(f).sel(time = plot_date)

        # force the data to be read into memory here
        tmp_xr.load()

    # merge the new data into the existing dataset
    if reanalysis_xr is None:
        reanalysis_xr = tmp_xr
    else:
        reanalysis_xr = xr.merge([reanalysis_xr, tmp_xr])

# display the contents of the dataset in the notebook
reanalysis_xr

In [ ]:
""" Cell 4: Make a map of 500 hPa geopotential height """

# *******************************
# Step 1: set the map projection
# *******************************

# set the map projection, centered on Bloomington, IN
map_projection = cartopy.crs.Orthographic(
    central_longitude = -86.5,
    central_latitude = 39.2,
)

# set the projection of the original data
data_projection = cartopy.crs.PlateCarree()

# **************************
# Step 2: create the figure
# **************************
# create the figure
figure = plt.figure(figsize = (5, 5))
# create the 'axes' (where the data will be plotted)
ax = figure.add_subplot(1, 1, 1, projection = map_projection)

# **********************
# Step 3: plot the data
# **********************
# select 500 hPa geopotential height
z500 = reanalysis_xr["hgt"].sel(level = 500)

# plot the data
z500.plot.contour(
    ax = ax,
    transform = data_projection,
    cmap = 'seismic',
    levels = 20,
    add_colorbar = True,
    )

# **************************
# Step 4: annotate the plot
# **************************
# add coastlines
ax.coastlines()

# add states
ax.add_feature(cartopy.feature.STATES)

# add grid lines
ax.gridlines()

# **********************
# Step 5: draw the plot
# **********************
plt.show()


In [ ]:
""" Cell 5: Plot a transect of 500 mb height anomalies and temperature anomalies, centered on the wave """

# check if temperature is in the dataset
if "air" not in reanalysis_xr:
    raise RuntimeError("Temperature is not in the dataset - skipping transect plot; uncomment in Cell 3 (see Cell 3 subset) to grab temperature.")

center_lat = 49 #
center_lon = -86.5 + 360 # Bloomington, IN
lon_span = 120 # degrees

# extract the desired latitude
transect_xr = reanalysis_xr.sel(lat = center_lat, method='nearest').sel(
    lon = slice(center_lon - lon_span / 2, center_lon + lon_span / 2))

# compute the 500 hPa height anomaly
z_anom = transect_xr["hgt"] - transect_xr["hgt"].mean(dim = "lon")
# add some metadata for the plot
z_anom.attrs['long_name'] = "Geopotential height anomaly"
z_anom.attrs['units'] = "m"

# compute temperature anomaly
t_anom = transect_xr["air"] - transect_xr["air"].mean(dim = "lon")
# add some metadata for the plot
t_anom.attrs['long_name'] = "Temperature anomaly"
t_anom.attrs['units'] = "K"


# create a figure
fig, axs = plt.subplots(figsize = (10,10), nrows = 2, sharex = True)

# plot the geopotential height anomaly in the first plot
z_anom.plot.contour(
    ax = axs[0],
    levels = 20,
    add_colorbar = True,
    )

# plot theta in the second plot
t_anom.plot.contour(
    ax = axs[1],
    levels = 20,
    add_colorbar = True,
    )


for ax in axs:
    # reverse the y-axis so 1000 mb is on bottom
    ax.invert_yaxis()

    # check if uwnd and omega are in the dataset
    # if so, plot vectors of wind speed and vertical velocity
    if "uwnd" in transect_xr:
        # convert wind from m/s to degrees/s
        Rearth = 6371000 # radius of the earth in meters
        u_dps = transect_xr["uwnd"] * 2*np.pi / (Rearth * np.cos(np.deg2rad(center_lat))) * 180/np.pi

        # convert omega from Pa/s to mb/s
        omega_mbps = transect_xr["omega"] / 100

        # plot the wind vectors
        ax.quiver(
            transect_xr.lon, # x-coordinates of the vectors
            transect_xr.level, # y-coordinates of the vectors
            u_dps, # u-component of the wind (in units of the transect)
            -omega_mbps, # vertical velocity (in units of the transect [add the negative sign b/c the axes are reversed])
        )


# show the plot
plt.show()
